<a href="https://colab.research.google.com/github/al34n1x/DataScience/blob/master/6.Gestion_de_datos/Gestion_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gestión de datos: Join, Combine, y Reshape

En muchas aplicaciones, los datos son distrbuidos a través de un diferentes archivos y base de datos, o en un formato que no es fácil de analizar. 
Para ello, utilizaremos herramientas que nos facilitarán el proceso de preparación de los datos. 
Algunas de ellas ya las hemos visto como parte de otros capítulos, por lo que destacaremos las más importantes.

## Indice Jerárquico

Es una herramienta importante de Pandas. Permite tener multiples índices en un mismo eje. Para ponerlo un poco más simple, te permite trabajar con datos de una dimensión superior en una inferiór (Ejemplo, ventas por local y a través del tiempo)

In [0]:
import pandas as pd
import numpy as np
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

Lo que está viendo es una vista predefinida de una Serie con un MultiIndex como índice. Los espacios en la visualización del índice significan "usar la etiqueta directamente arriba".
Con este tipo de índices puedes realizar lo que se llama partial-index, lo que permite obtener un subset de los datos.

In [0]:
data['b']

In [0]:
data.loc[:, 2] # La selección es posible incluso desde dentro del nivel

Los índices jerárquicos juegan un rol importante en el modelado y agrupamiento de datos. Por ejemplo, podemos reordenar los datos anteriores en un Dataframe usando el método **unstack**

In [0]:
data.unstack()

In [0]:
data.unstack().stack() # Es la función inversa

## Reordenando los diferentes niveles
Con los Dataframe, los ejes pueden tener índice jerárquicos también. Veamos el siguiente ejemplo:

In [0]:
df = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Marty', 'Marty', 'Doc'],
                              ['Lorraine', 'George', 'Delorean']])
df

In [0]:
df.index.names = ['key1', 'key2'] # Los niveles pueden tener nombres. 
df

In [0]:
df['Marty'] # Podemos seleccionar datos parciales

In [0]:
df.swaplevel('key1','key2') # También es posible reordenar por niveles

Finalmente, podemos aplicar funciones estasdísticas en Dataframes o series, como agregación en un eje particular. 
Considerando el ejemplo anterior, podemos realizar una agregación por nivel, ya sea por fila o columna: 

In [0]:
df.sum(level='key2')

## Indexing columnas en un Dataframe
Es muy común que algunas veces desees mover algunos índices de columnas a filas, o viceversa. Para ello podemos utilizar la función **set_index**

In [0]:
df = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
df

In [0]:
df2 = df.set_index(['c', 'd']) # Si agregamos el parámetro drop = false las columnas no se eliminaran del df
df2

# Combinando datasets

Los datos contenidos en los objetos pandas se pueden combinar de varias maneras:

**pandas.merge** conecta filas en DataFrames en función de una o más claves. Esto será familiar para los usuarios de SQL u otras bases de datos relacionales, ya que implementa operaciones de unión de bases de datos.

**pandas.concat** concatena o "apila" objetos juntos a lo largo de un eje.

El método de instancia **combine_first** permite unir datos superpuestos para completar los valores faltantes en un objeto con valores de otro.

## Database-Style joins en Dataframes

Las operaciones de merge o join combinan conjuntos de datos al vincular filas con una o más claves. Estas operaciones son centrales para las bases de datos relacionales. La función merge en pandas es el principal punto de entrada para usar estos algoritmos en sus datos.

In [0]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
print(df1)
print('\n')
print(df2)

Este es un ejemplo de una unión de muchos a uno; los datos en df1 tienen varias filas etiquetadas con a y b, mientras que df2 tiene solo una fila para cada valor en la columna clave. Llamando a fusionar con estos objetos obtenemos:  

In [0]:
pd.merge(df1,df2)

Puedes observar que no especifiqué en qué columna unir. Si no se especifica esa información, la combinación usa los nombres de columna superpuestos como las claves. Sin embargo, es una buena práctica especificar explícitamente:

In [0]:
pd.merge(df1, df2, on='key')

Como en las sentencias SQL, si los nombres de las columnas son diferentes en cada objeto, se puede especificar de forma separada.

In [0]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
print (df3)
print('\n')
print(df4)

In [0]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

Puedes notar que los valores **c** y **d** y los datos asociados faltan en el resultado. Por defecto, merge hace una unión 'interna' (inner join); Las claves en el resultado son la intersección, o el conjunto común que se encuentra en ambas tablas. Otras opciones posibles son **izquierda**, **derecha** y **exterior**. La unión externa toma la unión de las claves, combinando el efecto de aplicar las uniones izquierda y derecha.
A continuación encontrarás un diagrama con de SQL joins con su correspondiente sentencia en ese lenguaje que te será de utilidad para el desarrollo de tus actividades

<img src = "https://i.pinimg.com/564x/42/48/72/424872ac0b25c05e117b521d55616551.jpg">


A continuación se detallan las opciones que se encuentran disponibles en Pandas con el compartamiento asociado

Opción | Comportamiento
-------|-------
**inner**| Utiliza solo la combinación de claves comunes para ambas tablas
**left** | Utiliza solo la combinación de claves encontradas en la tabla declarada a izquierda
**right** | Utiliza solo la combinación de claves encontradas en la tabla declarada a derecha
**outter** | Utiliza solo la combinación de claves observada en ambas tablas juntas 

In [0]:
pd.merge(df1, df2, how='outer')

Para realizar merge con multiples keys, debemos pasar una lista de nombres de columnas

In [0]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

###Argumentos de la función **Merge**

A continuación se detallan los argumentos más utilizados con la función **merge** asociados a su descripción.

Argumento | Descripción
---------|------------
left | Dataframe se fusiona en el lado izquierdo
right | Data frame se fusiona en el lado derecho
how | con parámetro 'inner', 'outer', 'left', o 'right'. Default 'inner'
on | La unión se hace en base a nombre de columnas. Deben estar presentes en ambos Dataframes
left_on| Se utilizan las columnas del Dataframe izquierdo como claves
right_on| Análogo al 'left_on'
left_index|Utiliza  el índice de fila en la izquierda como clave del join.
right_index|Análogo al 'left_index'
sort | Ordena datos fusionados lexicográficamente por las claves.
suffixes|Tupla de valores de cadenas a agregar a una coluna en caso de sobreposición. Si por ejemplo tenemos data en ambos dataframes podemos agregar data_x, data_y como sufijos
copy|Si es falso, evita copiar datos en la estructura resultante.
indicator|Agrega una columna especial '_merge' que indica la fuente de cada fila. Valores pueden ser 'left_only', 'right_only', o, 'both'.


